In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [3]:
path = "../training/conf/network/conv_transformer.yaml"

In [12]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [13]:
cfg

{'_target_': 'text_recognizer.networks.ConvTransformer', 'input_dims': [1, 1, 576, 640], 'hidden_dim': 128, 'num_classes': 58, 'pad_index': 3, 'encoder': {'_target_': 'text_recognizer.networks.convnext.ConvNext', 'dim': 16, 'dim_mults': [2, 4, 8], 'depths': [3, 3, 6], 'downsampling_factors': [[2, 2], [2, 2], [2, 2]]}, 'decoder': {'_target_': 'text_recognizer.networks.transformer.Decoder', 'dim': 128, 'depth': 10, 'block': {'_target_': 'text_recognizer.networks.transformer.decoder_block.DecoderBlock', 'self_attn': {'_target_': 'text_recognizer.networks.transformer.Attention', 'dim': 128, 'num_heads': 12, 'dim_head': 64, 'dropout_rate': 0.2, 'causal': True, 'rotary_embedding': {'_target_': 'text_recognizer.networks.transformer.RotaryEmbedding', 'dim': 64}}, 'cross_attn': {'_target_': 'text_recognizer.networks.transformer.Attention', 'dim': 128, 'num_heads': 12, 'dim_head': 64, 'dropout_rate': 0.2, 'causal': False}, 'norm': {'_target_': 'text_recognizer.networks.transformer.RMSNorm', 'dim

In [14]:
net = instantiate(cfg)

In [15]:
from torchinfo import summary

In [16]:
summary(net, ((1, 1, 56, 1024), (1, 89)), device="cpu", depth=4)

Layer (type:depth-idx)                                  Output Shape              Param #
ConvTransformer                                         [1, 58, 89]               --
├─ConvNext: 1-1                                         [1, 128, 7, 128]          1,051,488
│    └─Conv2d: 2-1                                      [1, 16, 56, 1024]         800
│    └─ModuleList: 2                                    --                        --
│    │    └─ModuleList: 3                               --                        --
│    │    │    └─ConvNextBlock: 4-1                     [1, 16, 56, 1024]         10,080
│    │    │    └─Downsample: 4-2                        [1, 32, 28, 512]          2,080
│    │    └─ModuleList: 3                               --                        --
│    │    │    └─ConvNextBlock: 4-3                     [1, 32, 28, 512]          38,592
│    │    │    └─Downsample: 4-4                        [1, 64, 14, 256]          8,256
│    │    └─ModuleList: 3             

In [22]:
summary(net, ((1, 1, 576, 640), (1, 682)), device="cpu", depth=4)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [EfficientNet: 1, Sequential: 2, ZeroPad2d: 3, Conv2d: 3, BatchNorm2d: 3, Mish: 3, MBConvBlock: 3, Depthwise: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, Mish: 6, SqueezeAndExcite: 4, Sequential: 5, Conv2d: 6, Mish: 6, Conv2d: 6, Pointwise: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, MBConvBlock: 3, InvertedBottleneck: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, Mish: 6, Depthwise: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, Mish: 6, SqueezeAndExcite: 4, Sequential: 5, Conv2d: 6, Mish: 6, Conv2d: 6, Pointwise: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, MBConvBlock: 3, InvertedBottleneck: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, Mish: 6, Depthwise: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, Mish: 6, SqueezeAndExcite: 4, Sequential: 5, Conv2d: 6, Mish: 6, Conv2d: 6, Pointwise: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, MBConvBlock: 3, InvertedBottleneck: 4, Sequential: 5, 